<a href="https://colab.research.google.com/github/neo4j-contrib/training/blob/master/data_science/AppliedGraphAlgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Graph Algorithms

In [ ]:
!pip install neo4j pandas matplotlib

In [ ]:
from neo4j.v1 import GraphDatabase, basic_auth
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Change the line of code below to use the IP Address, Bolt Port, and Password of your Sandbox.
#driver = GraphDatabase.driver("bolt://<IP Address>:<Bolt Port>", auth=basic_auth("neo4j", "<Password>"))

driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "neo"))

## Betweenness Centrality

Betweenness centrality identifies nodes that are strategically positioned in the network, meaning that information will often travel through that person. Such an intermediary position gives that person power and influence.

Betweenness centrality is a raw count of the number of short paths that go through a given node. For example, if a node is located on a bottleneck between two large communities, then it will have high betweenness.

In [ ]:
query = """\
CALL algo.betweenness.stream("Character", "INTERACTS1", {direction: "BOTH"})
YIELD nodeId, centrality
MATCH (c:Character) WHERE ID(c) = nodeId
RETURN c.name, centrality
ORDER BY centrality DESC
LIMIT 10
"""

with driver.session() as session:
    result = session.run(query)
df = pd.DataFrame([dict(record) for record in result])    

In [ ]:
df.sort_values(by=["centrality"], ascending=False).head()

In [ ]:
query = """\
CALL algo.betweenness.stream("Character", "INTERACTS1", {direction: "BOTH"})
YIELD nodeId, centrality
MATCH (c:Character) WHERE ID(c) = nodeId
WITH c, centrality, [(c)-[r:INTERACTS1]-(other) | {character: other.name, weight: r.weight}] AS interactions
RETURN c.name, centrality,
       apoc.coll.sum([i in interactions | i.weight]) AS totalInteractions,
       [i in apoc.coll.reverse(apoc.coll.sortMaps(interactions, 'weight'))[..5] | i.character] as charactersInteractedWith
ORDER BY centrality DESC
LIMIT 10
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])  

In [ ]:
df.sort_values(by=["centrality"], ascending=False).head(10)

In [ ]:
df.sort_values(by=["totalInteractions"], ascending=False).head(10)

## Storing betweenness centrality

Although the betweenness centrality algorithm runs very quickly on this dataset we wouldn’t usually be running this types of algorithms in the normal request/response flow of a web/mobile app. Instead of that we can store the result of the calculation as a property on the node and then refer to it in future queries.

Each of the algorithms has a variant that saves its output to the database rather than returning a stream. Let’s run the betweenness centrality algorithm and store the result as a property named `book1BetweennessCentrality`:

In [ ]:
query = """\
CALL algo.betweenness("Character", "INTERACTS1", {direction: "BOTH", writeProperty: "book1BetweennessCentrality"})
"""
with driver.session() as session:
    session.run(query)

We can write the following query to find the most influential characters:

In [ ]:
query = """\
MATCH (c:Character)
RETURN c.name, c.book1BetweennessCentrality AS centrality
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])  

In [ ]:
df.sort_values(by=["centrality"], ascending=False).head()

## Exercise: Betweenness Centrality for books 2-5

Now we want to calculate the betweenness centrality for the other books in the series and store the results in the database.

* Write queries that call algo.betweenness for the INTERACTS2, INTERACTS3, and INTERACTS45 relationship types.

After you’ve done that see if you can write queries to answer the following questions:

* Which character had the biggest increase in influence from book 1 to 5?

Wh* ich character had the biggest decrease?

Bonus question:

* Which characters who were in the top 10 influencers in book 1 are also in the top 10 influencers in book 5?

## Page Rank

This is another version of weighted degree centrality with a feedback loop. This time, you only get your “fair share” of your neighbor’s importance.

i.e. your neighbor’s importance is split between their neighbors, proportional to the number of interactions with that neighbor.

Intuitively, PageRank captures how effectively you are taking advantage of your network contacts. In our context, PageRank centrality nicely captures narrative tension. Indeed, major developments occur when two important characters interact.

In [ ]:
with driver.session() as session:
    result = session.run("""CALL algo.pageRank('Character', 'INTERACTS1', 
                              {direction: 'BOTH', writeProperty:'book1PageRank'})""")
    print(result.peek())
    
    result = session.run("""CALL algo.pageRank('Character', 'INTERACTS2', 
                              {direction: 'BOTH', writeProperty:'book2PageRank'})""")
    print(result.peek()) 
    
    result = session.run("""CALL algo.pageRank('Character', 'INTERACTS3', 
                              {direction: 'BOTH', writeProperty:'book3PageRank'})""")
    print(result.peek())     
    
    result = session.run("""CALL algo.pageRank('Character', 'INTERACTS45', 
                              {direction: 'BOTH', writeProperty:'book45PageRank'})""")
    print(result.peek())         

In [ ]:
query = """\
MATCH (c:Character)
WITH c, [(c)-[r:INTERACTS1]-(other) | {character: other.name, weight: r.weight}] AS interactions
RETURN c.name, c.book1PageRank AS pageRank, c.book1BetweennessCentrality as centrality, 
       apoc.coll.sum([i in interactions | i.weight]) AS totalInteractions
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])      

In [ ]:
df.sort_values(by=["centrality"], ascending = False).head(10)

In [ ]:
df.sort_values(by=["pageRank"], ascending = False).head(10)

You’ll notice that there are some characters who have a high page rank but a very low betweenness centrality score.

This suggests that they aren’t necessarily influential in their own right, but are friends with important people. Varys is a good example of a character that fits this profile.

## Community Detection

We can detect communities in our data by running an algorithm which traverses the graph structure to find highly connected subgraphs with fewer connections other other subgraphs.

Run the following query to calculate the communities that exist based on interactions across all the books.

In [ ]:
query = """\
CALL algo.labelPropagation(
  'MATCH (c:Character) RETURN id(c) as id',
  'MATCH (c:Character)-[rel]->(c2) RETURN id(c) as source, id(c2) as target, SUM(rel.weight) as weight',
  'OUTGOING',
  {graph:'cypher', partitionProperty: 'community', iterations: 10})
"""

with driver.session() as session:
    result = session.run(query)
    print(result.peek())

In [ ]:
query = """\
MATCH (c:Character)
WHERE exists(c.community)
RETURN c.community AS community, count(*) AS count
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])   

In [ ]:
df.sort_values(by=["count"], ascending = False).head(10)

## Querying Communities

It’d be good to know who are the influential people in each community. To do that we’ll need to calculate a Page Rank score for each character across all the books:

In [ ]:
query = """\
CALL algo.pageRank(
  'MATCH (c:Character) RETURN id(c) as id',
  'MATCH (c:Character)-[rel]->(c2) RETURN id(c) as source,id(c2) as target, SUM(rel.weight) as weight',
  {graph:'cypher', writeProperty: 'pageRank', iterations: 10})
"""

with driver.session() as session:
    result = session.run(query)
    print(result.peek())

In [ ]:
query = """\
MATCH (c:Character)
WHERE exists(c.community)
WITH c ORDER BY c.pageRank DESC
RETURN c.community as cluster, count(*) AS count, collect(c.name)[0] AS mostInfluential
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])   

In [ ]:
df.sort_values(by=["count"], ascending = False).head(10)

## Intra-community Page Rank

We can also calculate the Page Rank within communities.

Run the following query to calculate the page rank for the 2nd largest community:

In [ ]:
query = """\
MATCH (c:Character) WHERE EXISTS(c.community)
WITH c.community AS communityId, COUNT(*) AS count
ORDER BY count DESC
SKIP 1 LIMIT 1
CALL apoc.cypher.doIt(
  "CALL algo.pageRank(
    'MATCH (c:Character) WHERE c.community =" + communityId + " RETURN id(c) as id',
    'MATCH (c:Character)-[rel]->(c2) WHERE c.community =" + communityId + " AND c2.community =" + communityId + " RETURN id(c) as source,id(c2) as target, sum(rel.weight) as weight',
    {graph:'cypher', writeProperty: 'communityPageRank'}) YIELD nodes RETURN count(*)", {})
YIELD value
RETURN value
"""

with driver.session() as session:
    result = session.run(query)
    print(result.peek())

In [ ]:
query = """\
MATCH (c:Character) WHERE EXISTS(c.community)
WITH c.community AS communityId, COUNT(*) AS count
ORDER BY count DESC
SKIP 1 LIMIT 1
MATCH (c:Character) WHERE c.community = communityId
RETURN c.name AS character, c.communityPageRank as pageRank
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])       

In [ ]:
df.sort_values(by=["pageRank"], ascending = False).head(10)

Let's now calculate the intra-community Page Rank for all the communities:

In [ ]:
query = """\
CALL algo.pageRank(
  'MATCH (c:Character) WHERE c.community=%d RETURN id(c) as id',
  'MATCH (c:Character)-[rel]->(c2) WHERE c.community=%d AND c2.community =%d RETURN id(c) as source,id(c2) as target, sum(rel.weight) as weight',
  {graph:'cypher', writeProperty: 'communityPageRank'}) 
YIELD nodes 
RETURN count(*)
"""

with driver.session() as session:
    for row in session.run("MATCH (c:Character) WHERE EXISTS(c.community) RETURN DISTINCT c.community AS communityId"):        
        community_id = row["communityId"]
        session.run(query % (community_id, community_id, community_id))
print("Page Ranks calculated")        

We can now work out who the most influential people are inside and outside a community:

In [ ]:
query = """\
MATCH (c:Character)
WHERE exists(c.community)
WITH c ORDER BY c.pageRank DESC
WITH  c.community as cluster, count(*) AS count, collect(c) AS characters
RETURN cluster, count, 
       apoc.coll.reverse(apoc.coll.sortNodes(characters, "pageRank"))[0].name AS overallInfluential,
       apoc.coll.reverse(apoc.coll.sortNodes(characters, "communityPageRank"))[0].name AS communityInfluential
ORDER BY count DESC
"""

with driver.session() as session:
    df = pd.DataFrame([dict(record) for record in session.run(query)])   

In [ ]:
df.sort_values(by=["count"], ascending = False).head(10)